# Linear Regression using PyTorch with custom data from scratch.

In [2]:
import torch
import numpy as np

In [3]:
# Create your training data

# (temp, rainfall, humidity)

inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [4]:
# Create your Targets

# (apples, oranges)

targets = np.array([[56, 70],
                   [81, 101], 
                   [119, 133],
                   [22, 37],
                   [103, 119]], dtype = 'float32')

In [5]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Set up your weights and biases

we have the equation as 

`y (Apples) = w11 * temp + w12 * rainfall + w13 * humidity + b1`

`y (Apples) = w21 * temp + w22 * rainfall + w23 * humidity + b2`

In [6]:
# cause we have 2 rows and 3 columns for our weights.
w = torch.randn(2, 3, requires_grad=True)
# cause we have 2 biases
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[-2.3197,  1.9741, -1.8741],
        [-0.7245, -1.4532, -1.1253]], requires_grad=True)
tensor([-0.1157,  0.8850], requires_grad=True)


`.randn()` uses Normal Distribution with mean 0 and standard deviation 1

In [7]:
# Define the model
def model(x):
    return x @ w.t() + b

Here, `@` represents matrix multiplication in pytorch and `.t()` represents transpose of a matrix.

In [8]:
# Generate Predictions
preds = model(inputs)
print(preds)

tensor([[-117.7744, -197.7606],
        [-157.4288, -264.9516],
        [ -46.0947, -322.1504],
        [-221.1807, -177.1424],
        [-101.8467, -267.3897]], grad_fn=<AddBackward0>)


In [9]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Loss Function

In [10]:
# Define Mean Squared Error Loss Function
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

This `.numel()` gives us the number of elements in the array.

In [11]:
# Compute Loss
loss = mse(preds, targets)
print(loss)

tensor(82333.3281, grad_fn=<DivBackward0>)


# Compute Gradients

In [12]:
loss.backward()

In [13]:
print(w)
print(w.grad)

tensor([[-2.3197,  1.9741, -1.8741],
        [-0.7245, -1.4532, -1.1253]], requires_grad=True)
tensor([[-17536.9297, -16973.8730, -11128.8379],
        [-28189.9238, -31487.0762, -19260.7754]])


Note that 

If a gradient element is positive:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

If a gradient element is negative:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

In [14]:
w
w.grad

tensor([[-17536.9297, -16973.8730, -11128.8379],
        [-28189.9238, -31487.0762, -19260.7754]])

In [15]:
with torch.no_grad():
    w -= w.grad * 1e-7
    b -= b.grad * 1e-7

`torch.no_grad()` holds or stops the gradients to store in the variable for a bit.

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [16]:
loss = mse(preds, targets)
print(loss)

tensor(82333.3281, grad_fn=<DivBackward0>)


we need to set the gradients back to zero or else Torch will accumulate it by computing new gradients and filling it again and again.

so we use `.zero_()` method to set gradients to zero.

In [17]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Using Gradient Descent altogether

steps to achive it 

* Generate predictions
*Calculate the loss
*Compute gradients w.r.t the weights and biases
*Adjust the weights by subtracting a small quantity proportional to the gradient
*Reset the gradients to z

In [19]:
preds = model(inputs)
print(preds)

tensor([[-117.4848, -197.2610],
        [-157.0486, -264.2946],
        [ -45.6501, -321.3715],
        [-220.8877, -176.6482],
        [-101.4848, -266.7581]], grad_fn=<AddBackward0>)


In [20]:
loss = mse(preds, targets)
print(loss)

tensor(82045.9219, grad_fn=<DivBackward0>)


In [21]:
# compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-17507.0723, -16941.9160, -11109.0879],
        [-28138.3203, -31431.5703, -19226.5352]])
tensor([-204.7112, -337.2667])


In [22]:
# adjust the weight element to decrese the weight derivative.
with torch.no_grad():
    w -= w.grad * 1e-7
    b -= b.grad * 1e-7
    w.grad.zero_()
    b.grad.zero_()

In [23]:
print(w)
print(b)

tensor([[-2.3162,  1.9775, -1.8719],
        [-0.7189, -1.4469, -1.1215]], requires_grad=True)
tensor([-0.1156,  0.8850], requires_grad=True)


In [25]:
# Calculate the loss again
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(81759.5234, grad_fn=<DivBackward0>)


# Train for multiple Epochs

In [37]:
for i in range(500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [38]:
# Calculate Loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1.5969, grad_fn=<DivBackward0>)


In [39]:
preds

tensor([[ 57.2644,  70.5063],
        [ 81.2627, 100.1848],
        [120.6806, 133.7482],
        [ 21.6394,  37.2244],
        [ 99.9354, 118.3488]], grad_fn=<AddBackward0>)

In [40]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Commit to Jovian

In [46]:
import jovian

In [47]:
jovian.commit(project='pytorch_linear_regression_from_scratch')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Creating a new project "hemanthhari2000/pytorch_linear_regression_from_scratch"
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/hemanthhari2000/pytorch-linear-regression-from-scratch


'https://jovian.ai/hemanthhari2000/pytorch-linear-regression-from-scratch'